In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import scanpy as sc
import anndata
from scipy import io
from scipy.sparse import coo_matrix, csr_matrix
import numpy as np
import os
import pandas as pd
from matplotlib.pyplot import rc_context

In [5]:
import milopy
import milopy.core as milo

In [6]:
sc.set_figure_params(scanpy=True, fontsize=12)

In [ ]:
mye=sc.read_h5ad('fetal_Myeloid_si_0321_new.h5ad')

In [ ]:
mye.obs['Age2_group'] = (
    mye.obs["Age2"]
    .map(lambda x: { '8Wk':8,
                     '9Wk':9,
                     '10Wk':10,
                     '11Wk':11, 
                     '12Wk':12, 
                     '13Wk':13, 
                     '14Wk':14, 
                     '16Wk':16,
                     '17Wk':17,
                     '18Wk':18, 
                     '19Wk':19,
                     '20Wk':20, 
                     '21Wk':21, 
                     '23Wk':23
                   }.get(x, x))
    .astype("int")
)

In [ ]:
d = 30
k = 50

sc.pp.neighbors(mye, n_neighbors=k, n_pcs=d)

In [ ]:
milo.make_nhoods(mye, prop=1) ###all cells

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
nhood_size = np.array(mye.obsm["nhoods"].sum(0)).ravel()
plt.hist(nhood_size, bins=100);

In [ ]:
milo.count_nhoods(mye, sample_col="sample_name")

In [ ]:
milo.DA_nhoods(mye, design="~Age2_group")

In [ ]:
import milopy.utils
milopy.utils.build_nhood_graph(mye)

In [ ]:
import milopy.plot as milopl

In [ ]:
plt.rcParams["figure.figsize"] = [5,5]
milopl.plot_nhood_graph(mye, 
                        alpha=0.01, ## SpatialFDR level (1%) 
                        min_size=2 ## Size of smallest dot
                       )

In [ ]:
milopy.utils.annotate_nhoods(mye, anno_col='celltype2')

In [ ]:
plt.hist(mye.uns['nhood_adata'].obs["nhood_annotation_frac"]);
plt.xlabel("celltype fraction")

In [ ]:
with rc_context({'figure.figsize': (8, 4)}):
    sc.pl.violin(mye.uns['nhood_adata'], "logFC", groupby="nhood_annotation", rotation=90, show=False,
                order=['LYVE1+CX3CR1+ MΦ','Cyling','CMP', 'classical monocytes', 'CX3CR1-CLEC10A_lo MΦ','CX3CR1+CLEC10A_hi MΦ','LYVE1+CX3CR1- MΦ','CX3CR1-CLEC10A_hi MΦ',
                      'cDC1', 'cDC2', 'pDC','MMP9_int MΦ', 'MMP9_hi MΦ','nonclassical monocytes','Migratory DCs',
       'Mast cells'], size=2,palette=col2);
    plt.axhline(y=0, color='black', linestyle='--');
    plt.show()